## Introduction

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from pprint import pprint

## Data exploration - Tree

### First, we load the raw JSON data

In [ ]:
filename = "../inputs/2022_MARCH.json"

with open(filename, "r") as jsonfile:
    json_data = json.loads(jsonfile.read())
    
print(str(json_data)[:512] + "...")

### Then, we create a tree with the data

In [ ]:
from json_ld_semantics.semantics import Tree
tree = Tree(json_data)

### We can explore whatever is inside

In [ ]:
tree

In [ ]:
tree.children[0]

In [ ]:
print(len(tree.get_paths()))
tree.get_paths()

In [ ]:
tree.get_children_from_path("$.timelineObjects[4].activitySegment.activities[0].activityType")

#### `get_children_from_path` supports wildcards and returns every match

In [ ]:
tree.get_children_from_path("$.timelineObjects[4].activitySegment.activities[*].*")

## Data semantics - Model

### We create a new model

In [ ]:
from json_ld_semantics.model import Model
model = Model()

### We can add files to a model to process it

In [ ]:
model.add_files(filename)
model.process_files()

### Or, if we already have a traversal, we can create the model directly from it

In [ ]:
model = Model(traversal=tree.export_traversal())

### Now, we can see what paths are available in our model

In [ ]:
model.get_paths()

### But more importantly, we can see what attributes are linked to these paths

In [ ]:
import pandas as pd

In [ ]:
liste = model.to_list()
df = pd.DataFrame(data=liste[1:], columns=liste[0])
df.head()

### It's a bit empty for now... Let's add some info.

In [ ]:
model.set_attribute(
    "$.timelineObjects",
    unique=True,
    description="List of semantic locations dertermined by Google. Can be either `placeVisit` or `activitySegment`."
)

In [ ]:
model.set_attribute(
    "$.timelineObjects[*]",
    unique=False,
    description="One of the semantic locations. Can be either `placeVisit` or `activitySegment`."
)

In [ ]:
model.set_attribute(
    "$.timelineObjects[*].placeVisit.location",
    descriptiveType="https://schema.org/location",
    unique=False,
    description="Location that Google thinks you visited (latitude, longitude, id, address, name and confidence)."
)

In [ ]:
liste = model.to_list()
df = pd.DataFrame(data=liste[1:], columns=liste[0])
df.head()

## Data semantics - Apply the model

### Now that we have a working model, let's apply it back to our existing tree

In [ ]:
tree.get_children_from_path("$.timelineObjects")[0]

In [ ]:
tree.apply_model(model)  # TODO

In [ ]:
tree.get_children_from_path("$.timelineObjects")[0]

## Data semantics - Export and import the model

### If you have an existing model, you can export it!

In [ ]:
model.dump_traversal(filename="my_model.pickle")

### Afterwards it can be imported as well.

In [ ]:
model.load_traversal(filename="my_model.pickle")